In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/Project-MONAI/MONAI.git

Cloning into 'MONAI'...
remote: Enumerating objects: 36959, done.
remote: Counting objects: 100% (213/213), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 36959 (delta 104), reused 129 (delta 62), pack-reused 36746
Receiving objects: 100% (36959/36959), 64.16 MiB | 39.50 MiB/s, done.
Resolving deltas: 100% (29658/29658), done.


In [ ]:
%cd MONAI/


/content/MONAI


In [ ]:
!pip install -e '.[all]'

Obtaining file:///content/MONAI


In [ ]:
model_dir = '/content/drive/MyDrive/template/our-git-project/Pretrained_Results_final_12_12_23'

In [ ]:
import os
import torch
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import OrderedDict
from monai.data import ArrayDataset, GridPatchDataset, create_test_image_3d, PatchIter
from monai.apps import download_url
from monai.utils import set_determinism
from monai.losses import DiceCELoss
from monai.inferers import sliding_window_inference
from monai.config import print_config
from monai.transforms import (
    AsDiscrete,
    Compose,
    CropForeground,
    EnsureChannelFirst,
    LoadImage,
    Orientation,
    RandFlip,
    RandCropByPosNegLabel,
    RandShiftIntensity,
    ScaleIntensityRange,
    RandRotate90,
    ToTensor,
)

from monai.metrics import DiceMetric, MeanIoU
from monai.networks.nets import UNETR

from monai.data import (
    DataLoader,
    CacheDataset,
)

print_config()

In [ ]:
set_determinism(seed=960)

In [ ]:
use_pretrained = True


if use_pretrained:

    dst = '/content/drive/MyDrive/template/our-git-project/Pretrained_Results_final_v4/best_model.pt'

    pretrained_path = os.path.normpath(dst)


In [ ]:
import os
from glob import glob
import shutil
from tqdm import tqdm
import numpy as np
import nibabel as nib
from monai.transforms import(
    Compose,
    EnsureChannelFirstd,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    NormalizeIntensityd,
    CropForegroundd,
    RandShiftIntensityd,
    RandFlipd,
    FgBgToIndicesd,
    RandRotate90d,
    RandCropByPosNegLabeld,
    Activations,



)
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism

# Training Hyper-params
lr = 1e-4
max_iterations = 10000
eval_num = 100

original_data_folder = '/content/drive/MyDrive/template/our-git-project/preprocessed_images/'
mask_data_folder = '/content/drive/MyDrive/template/our-git-project/preprocessed_images/'

# List of image file paths
images  = sorted(glob(os.path.join(original_data_folder, '*_orig.nii.gz')))
segs  = sorted(glob(os.path.join(mask_data_folder, '*_masks.nii.gz')))

 # define transforms for image and segmentation
train_imtrans = Compose(
        [
            LoadImage(),
            EnsureChannelFirst(),
            Orientation(axcodes="RAS"),
            RandFlip(spatial_axis=[0], prob=0.3,),
            RandFlip(spatial_axis=[1],prob=0.3,),
            RandFlip(spatial_axis=[2],prob=0.3,),
            RandRotate90(prob=0.5, max_k=3, spatial_axes=(0, 2)),
            ToTensor(),
        ]
    )
train_segtrans = Compose(
        [
            LoadImage(),
            EnsureChannelFirst(),
            Orientation(axcodes="RAS"),
            RandFlip(spatial_axis=[0], prob=0.3,),
            RandFlip(spatial_axis=[1],prob=0.3,),
            RandFlip(spatial_axis=[2],prob=0.3,),
            RandRotate90(prob=0.5, max_k=3, spatial_axes=(0, 2)),
            ToTensor(),
        ]
    )

val_imtrans = Compose([LoadImage(),
            EnsureChannelFirst(),
            Orientation(axcodes="RAS"),
            ToTensor(),])
val_segtrans = Compose([LoadImage(),
            EnsureChannelFirst(),
            Orientation(axcodes="RAS"),
            ToTensor(),])




In [ ]:
from monai.utils import first
# Define nifti dataset, dataloader
train_ds = ArrayDataset(images[:-17], train_imtrans, segs[:-17], train_segtrans)
train_loader1 = DataLoader(val_ds, batch_size=5, num_workers=8, pin_memory=torch.cuda.is_available())

val_ds = ArrayDataset(images[-17:], val_imtrans, segs[-17:], val_segtrans)
val_loader1 = DataLoader(val_ds, batch_size=5, num_workers=8, pin_memory=torch.cuda.is_available())

patch_iter = PatchIter(patch_size=(128, 128, 128), start_pos=(0, 0, 0))


def img_seg_iter(x):
    for img, seg in zip(patch_iter(x[0]), patch_iter(x[1])):
        print(img[1], seg[1]) #print coordinates
        yield ((img[0], seg[0]), )

train_ds1 = GridPatchDataset(train_ds, img_seg_iter, with_coordinates=False)
val_ds1 = GridPatchDataset(val_ds, img_seg_iter, with_coordinates=False)

train_loader = torch.utils.data.DataLoader(
    train_ds1, batch_size=2, num_workers=8, pin_memory=torch.cuda.is_available()
)
val_loader = torch.utils.data.DataLoader(
    val_ds1, batch_size=2, num_workers=8, pin_memory=torch.cuda.is_available()
)
im, seg = first(loader)
print("image shapes:", im.shape, seg.shape


In [ ]:
from monai.networks.nets import UNet
from monai.networks.layers import Act, Norm
from monai.losses import DiceLoss, DiceCELoss
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = UNETR(
    in_channels=1,
    out_channels=2,
    img_size=(48, 48, 48),
    feature_size=16,
    hidden_size=768,
    mlp_dim=3072,
    num_heads=12,
    pos_embed="conv",
    norm_name="instance",
    res_block=True,
    dropout_rate=0.0,
).to(device)

# Load ViT backbone weights into UNETR
if use_pretrained is True:
    print("Loading Weights from the Path {}".format(pretrained_path))
    vit_dict = torch.load(pretrained_path)
    vit_weights = vit_dict["state_dict"]

    # Remove items of vit_weights if they are not in the ViT backbone (this is used in UNETR).
    # For example, some variables names like conv3d_transpose.weight, conv3d_transpose.bias,
    # conv3d_transpose_1.weight and conv3d_transpose_1.bias are used to match dimensions
    # while pretraining with ViTAutoEnc and are not a part of ViT backbone.
    model_dict = model.vit.state_dict()

    vit_weights = {k: v for k, v in vit_weights.items() if k in model_dict}
    model_dict.update(vit_weights)
    model.vit.load_state_dict(model_dict)
    del model_dict, vit_weights, vit_dict
    print("Pretrained Weights Succesfully Loaded !")

elif use_pretrained is False:
    print("No weights were loaded, all weights being used are randomly initialized!")

model.to(device)

loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
torch.backends.cudnn.benchmark = True
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-5)

post_label = AsDiscrete(to_onehot=2)
post_pred = AsDiscrete(argmax=True, to_onehot=2)
dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
IoU_metric = MeanIoU(include_background=True, reduction="mean", get_not_nans=False)
global_step = 0
dice_val_best = 0.0
IoU_val_best = 0.0
global_step_best = 0
epoch_loss_values = []
metric_values = []
IoU_values = []

In [ ]:
import os
import torch
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import OrderedDict

from monai.apps import download_url
from monai.utils import set_determinism
from monai.losses import DiceCELoss
from monai.inferers import sliding_window_inference
from monai.config import print_config
from monai.transforms import (
    AsDiscrete,
    Compose,
    CropForegroundd,
    EnsureChannelFirstd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
    ToTensord,
    Activations,
)

from monai.metrics import DiceMetric
from monai.networks.nets import UNETR

from monai.data import (
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)

print_config()

In [ ]:
def validation(epoch_iterator_val):
    model.eval()
    dice_vals = []
    IoU_vals = []
    with torch.no_grad():
          for _step, batch in enumerate(epoch_iterator_val):
                  val_inputs, val_labels = (batch["vol"].cuda(), batch["seg"].cuda())
                  val_outputs = sliding_window_inference(val_inputs, (48, 48, 48), 4, model)
                  val_labels_list = decollate_batch(val_labels)
                  val_labels_convert = [post_label(val_label_tensor) for val_label_tensor in val_labels_list]
                  val_outputs_list = decollate_batch(val_outputs)
                  val_output_convert = [post_pred(val_pred_tensor) for val_pred_tensor in val_outputs_list]
                  dice_metric(y_pred=val_output_convert, y=val_labels_convert)
                  dice = dice_metric.aggregate().item()
                  dice_vals.append(dice)
                  IoU_metric(y_pred=val_output_convert, y=val_labels_convert)
                  IoU = IoU_metric.aggregate().item()
                  IoU_vals.append(IoU)
                  epoch_iterator_val.set_description("Validate (%d / %d Steps) (dice=%2.5f)" % (global_step, 10.0, dice))
                  epoch_iterator_val.set_description("Validate (%d / %d Steps) (IoU=%2.5f)" % (global_step, 10.0, IoU))

          dice_metric.reset()
          IoU_metric.reset()

    mean_dice_val = np.mean(dice_vals)
    mean_IoU_val = np.mean(IoU_vals)

    return mean_dice_val,mean_IoU_val


def train(global_step, train_loader, dice_val_best, IoU_val_best, global_step_best):
    model.train()
    epoch_loss = 0
    step = 0
    epoch_iterator = tqdm(train_loader, desc="Training (X / X Steps) (loss=X.X)", dynamic_ncols=True)
    for step, batch in enumerate(epoch_iterator):
              step += 1
              x, y = (batch["vol"].cuda(), batch["seg"].cuda())
              logit_map = model(x)
              loss = loss_function(logit_map, y)
              loss.backward()
              epoch_loss += loss.item()
              optimizer.step()
              optimizer.zero_grad()
              epoch_iterator.set_description("Training (%d / %d Steps) (loss=%2.5f)" % (global_step, max_iterations, loss))

              if (global_step % eval_num == 0 and global_step != 0) or global_step == max_iterations:
                  epoch_iterator_val = tqdm(val_loader, desc="Validate (X / X Steps) (IoU=X.X)", dynamic_ncols=True)
                  dice_val, IoU_val = validation(epoch_iterator_val)

                  epoch_loss /= step
                  epoch_loss_values.append(epoch_loss)
                  metric_values.append(dice_val)
                  IoU_values.append(IoU_val)
                  if dice_val > dice_val_best:
                      dice_val_best = dice_val
                      global_step_best = global_step
                      torch.save(model.state_dict(), os.path.join(model_dir, "best_metric_model.pth"))
                      print(
                          "Model Was Saved ! Current Best Avg. dice: {} Current Avg. dice: {}".format(dice_val_best, dice_val)
                      )
                  else:
                      print(
                          "Model Was Not Saved ! Current Best Avg. dice: {} Current Avg. dice: {}".format(
                              dice_val_best, dice_val
                          )
                      )
                  if IoU_val > IoU_val_best:
                      IoU_val_best = IoU_val
                      print(
                          "Model Was Saved ! Current Best Avg. IoU: {} Current Avg. IoU: {}".format(IoU_val_best, IoU_val)
                      )
                  else:
                      print(
                          "Model Was Not Saved ! Current Best Avg. IoU: {} Current Avg. IoU: {}".format(
                              IoU_val_best, IoU_val
                          )
                      )


                  plt.figure(1, (12, 6))
                  plt.subplot(1, 2, 1)
                  plt.title("Iteration Average Loss")
                  x = [eval_num * (i + 1) for i in range(len(epoch_loss_values))]
                  y = epoch_loss_values
                  plt.xlabel("Iteration")
                  plt.plot(x, y)
                  plt.grid()
                  plt.subplot(1, 2, 2)
                  plt.title("Val Mean Dice")
                  x = [eval_num * (i + 1) for i in range(len(metric_values))]
                  y = metric_values
                  plt.xlabel("Iteration")
                  plt.plot(x, y)
                  plt.grid()
                  plt.savefig(os.path.join(model_dir, "unet_finetune_quick_update.png"))
                  plt.clf()
                  plt.close(1)

              global_step += 1
    return global_step, dice_val_best, IoU_val_best, global_step_best


while global_step < max_iterations:
    global_step, dice_val_best, IoU_val_best, global_step_best = train(global_step, train_loader, dice_val_best, IoU_val_best, global_step_best)
model.load_state_dict(torch.load(os.path.join(model_dir, "best_metric_model.pth")))

print(f"train completed, best_metric: {dice_val_best:.4f} " f"at iteration: {global_step_best}")

In [ ]:
plt.figure(1, (12, 6))
plt.subplot(1, 2, 1)
plt.title("Iteration Average Loss")
x = [eval_num * (i + 1) for i in range(len(epoch_loss_values))]
y = epoch_loss_values
plt.xlabel("Iteration")
plt.plot(x, y)
plt.grid()
plt.subplot(1, 2, 2)
plt.title("Val Mean Dice")
x = [eval_num * (i + 1) for i in range(len(metric_values))]
y = metric_values
plt.xlabel("Iteration")
plt.plot(x, y)
plt.grid()
plt.show()

In [ ]:
from glob import glob
original_data_folder = '/content/drive/MyDrive/template/our-git-project/preprocessed_images/'
mask_data_folder = '/content/drive/MyDrive/template/our-git-project/preprocessed_images/'

# List of image file paths
vol  = sorted(glob(os.path.join(original_data_folder, '*_orig.nii.gz')))
segs  = sorted(glob(os.path.join(mask_data_folder, '*_masks.nii.gz')))


data_dicts = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(vol, segs)]
train_files, test_files = data_dicts[:-25], data_dicts[-25:]


test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            EnsureChannelFirstd(keys=["vol", "seg"]),
            #Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            #NormalizeIntensityd(keys="vol", nonzero=True, channel_wise=True),
            CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            RandCropByPosNegLabeld(
            keys=["vol", "seg"],
            label_key="seg",
            spatial_size=(128, 128, 128),
            pos=4,
            neg=1,
            num_samples=10,
            image_key="vol",
            image_threshold=0,
            ),
            ToTensord(keys=["vol", "seg"]),


        ]
    )

test_ds = CacheDataset(data=test_files, transform=test_transforms, cache_rate=1.0)
test_loader = DataLoader(test_ds, batch_size=1)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = UNETR(
    in_channels=1,
    out_channels=2,
    img_size=(48, 48, 48),
    feature_size=16,
    hidden_size=768,
    mlp_dim=3072,
    num_heads=12,
    pos_embed="conv",
    norm_name="instance",
    res_block=True,
    dropout_rate=0.0,
).to(device)


In [ ]:
model.load_state_dict(torch.load(
    os.path.join(model_dir, "best_metric_model.pth")))
model.eval()

In [ ]:
from monai.utils import first
sw_batch_size = 10
roi_size = (48, 48, 48)
with torch.no_grad():
    test_patient = first(test_loader)
    t_volume = test_patient['vol'].cuda()
    val_labels = test_patient['seg'].cuda()

    val_outputs = sliding_window_inference(t_volume.to(device), roi_size, sw_batch_size, model)
    val_labels_list = decollate_batch(val_labels)
    val_labels_convert = [post_label(val_label_tensor) for val_label_tensor in val_labels_list]
    val_outputs_list = decollate_batch(val_outputs)
    val_output_convert = [post_pred(val_pred_tensor) for val_pred_tensor in val_outputs_list]
    dice_metric(y_pred=val_output_convert, y=val_labels_convert)
    dice = dice_metric.aggregate().item()
    IoU_metric(y_pred=val_output_convert, y=val_labels_convert)
    IoU = IoU_metric.aggregate().item()
    print("Test Dice Value: ",dice)
    print("Test IoU Value: ",IoU)

In [ ]:
model.load_state_dict(torch.load(
    os.path.join(model_dir, "best_metric_model.pth")))
model.eval()
with torch.no_grad():
        test_patient = first(test_loader)
        t_volume = test_patient['vol']
        #t_segmentation = test_patient['seg']

        test_outputs = sliding_window_inference(t_volume.to(device), roi_size, sw_batch_size, model)
        sigmoid_activation = Activations(sigmoid=True)
        test_outputs = sigmoid_activation(test_outputs)
        test_outputs = test_outputs > 0.5

        for i in range(45):
            # plot the slice [:, :, 80]
            plt.figure("check", (20, 4))
            plt.subplot(1, 7, 1)
            plt.title(f"image {i}")
            plt.imshow(test_patient['vol'].detach().cpu()[0, 0, :, :, i], cmap="gray")
            plt.subplot(1, 7, 2)
            plt.title(f"argmax {i}")
            argmax = [AsDiscrete(argmax=True)(j) for j in decollate_batch(test_outputs)]
            plt.imshow(argmax[0].detach().cpu()[0, :, :, i])
            plt.subplot(1, 7, 3)
            plt.title(f"largest {i}")
            largest = [KeepLargestConnectedComponent(applied_labels=[1])(j) for j in argmax]
            plt.imshow(largest[0].detach().cpu()[0, :, :, i])
            plt.subplot(1, 7, 4)
            plt.title(f"contour {i}")
            contour = [LabelToContour()(j) for j in largest]
            plt.imshow(contour[0].detach().cpu()[0, :, :, i])
            plt.subplot(1, 7, 5)
            plt.title(f"map image {i}")
            map_image = contour[0] + t_volume.to(device)[0]
            plt.imshow(map_image.detach().cpu()[0, :, :, i], cmap="gray")
            plt.subplot(1, 7, 6)
            plt.title(f"output {i}")
            plt.imshow(test_outputs_1.detach().cpu()[0, 1, :, :, i])
            plt.subplot(1, 7, 7)
            plt.title(f"label {i}")
            plt.imshow(test_patient["seg"][0, 0, :, :, i] != 0)
            plt.show()

In [ ]:
model.load_state_dict(torch.load(
    os.path.join(model_dir, "best_metric_model.pth")))
model.eval()
with torch.no_grad():
    for i, val_data in enumerate(test_loader):
      for j in range(48):
        roi_size = (48, 48, 48)
        sw_batch_size = 10
        val_data = val_data["vol"].to(device)
        val_output = sliding_window_inference(val_data, roi_size, sw_batch_size, model)
        plt.figure("check", (20, 4))
        plt.subplot(1, 5, 1)
        plt.title(f"image {i}")
        plt.imshow(val_data.detach().cpu()[0, 0, :, :, j], cmap="gray")
        plt.subplot(1, 5, 2)
        plt.title(f"argmax {i}")

        argmax = [AsDiscrete(argmax=True)(i) for i in decollate_batch(val_output)]
        plt.imshow(argmax[0].detach().cpu()[0, :, :, j])
        plt.subplot(1, 5, 3)
        plt.title(f"largest {i}")
        largest = [KeepLargestConnectedComponent(applied_labels=[1])(i) for i in argmax]
        plt.imshow(largest[0].detach().cpu()[0, :, :, j])
        plt.subplot(1, 5, 4)
        plt.title(f"contour {i}")
        contour = [LabelToContour()(i) for i in largest]
        plt.imshow(contour[0].detach().cpu()[0, :, :, j])
        plt.subplot(1, 5, 5)
        plt.title(f"map image {i}")
        map_image = contour[0] + val_data[0]
        plt.imshow(map_image.detach().cpu()[0, :, :, j], cmap="gray")

        plt.show()